In [27]:
import random
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import math

In [ ]:
####This is poor generator stuff, you can ignore 

shrink = np.array([[1,0,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0],
              [0,0,0,0,0,1,0,0],
              [0,0,0,0,0,0,0,1]])
q = np.linalg.inv(np.array([[1,-3,-3,-3],
              [-3,1,-3,-3],
              [-3,-3,1,-3],
              [-3,-3,-3,1]]))

#take transpose of row reduced gram matrix to get linear relations 
#pivot columns work 

linrel = 1/2 * np.array([[2,0,0,0],
              [1,1,-1,1],
              [0,2,0,0],
              [1,1,1,-1],
              [-1,1,1,1],
              [0,0,2,0],
              [1,-1,1,1],
              [0,0,0,2]])
alpha = np.array([2*math.sqrt(2),2*math.sqrt(2),2*math.sqrt(2),2*math.sqrt(2),0,0,0,0])

wtpw = np.matmul(linrel,np.matmul(q,shrink))
i = np.identity(8)

curv = np.array([-1,2,7,4,11,8,3,6])

print(i - 2* np.matmul(np.outer(alpha,alpha),wtpw)/abs(np.matmul(alpha,np.matmul(wtpw,alpha))))

print(np.matmul((i - 2*np.matmul(np.outer(alpha,alpha),wtpw)/abs(np.matmul(alpha,np.matmul(wtpw,alpha)))), curv))



In [ ]:
##This is just a demonstration of how it might walk over the words, can ignore unless you want to see that, not used in the actual calculation.

word = [1]
while word:
  print(word)
  if random.uniform(0, len(word))<1:
    word = over(word)
  else:
    word = down(word)

In [30]:
##These are the main functions used for 


#function that returns the new curvature for the nth generator, just for checking whether we should go deeper into the word tree or start going down 
def transval(n,root):
  return 2*sum(root)-3*root[n-1]

#function for when the word would have given a curvature that is too big 
def down(word): 
  #if not word:
    #return word #this is older stuff you can probably ignore

  if word[-1] == 4:   #if the word ends in the last generator, drop that and repeat this whole function with that new word
    word.pop()
    word = down(word)
  elif len(word)>1 and word[-1]+1==word[-2]: #if the word has length 2 and the last two generators are n+1, n
    if word[-1]==3: #if the word ended in "43", then drop those two (since that is as far as we could go in this part of the tree), and if it is nonempty, then call this whole function again 
      word.pop()
      word.pop()
      if word: 
        word = down(word)
    else: #if it ended in somehting like "21", then that becomes "23", like it skips over "22," since that wouldn't be reduced.
      word[-1]=word[-1]+2
  else: #if it wasn't one of the weird above cases then just change the last generator to the one after that 
    word[-1]=word[-1]+1
  return word

#function for when the word gives a curvature that's still under the bound
#just adds a 1 or 2 at the end of the word, which is basically like the left-most branch above the word in the tree
def over(word):
  if word[-1] == 1:
    word.append(2)
  else:
    word.append(1)
  return word

In [31]:
#this is the function we want to fit the curve to
def func(x, a, b):
    return a * x**b

#this function fits the curve and doesn't actually return anything but prints the fractal dimension
#takes in a number of how many times we want to divide , the bound from below, and the list of curvatures under that bound
def curvefit(n, bound, curvatures):
  x = []
  y = []
  for j in range(n): #n is just how many sub-bounds we want to have, so if our bound was like curvatures under 30,000, and we had n = 30
                      #then we would be counting like curvatures under 1000, 2000, 3000, ... 30,0000. 
    x.append(int((j+1)*bound/n))
    y.append(sum(i < (j+1)*bound/n for i in circlelist)) #this is just counting the number of curvatures under each of our sub-bounds 
  popt,pcov=curve_fit(func,x,y) #curve fit to our list of like [1000,2000,...,30000] matched with the count of circles under each of those bounds
  print("fractal dimension: " + str(popt[1])) #popt is like the [a,b] that is the best curve fit for ax^b, so we just take b 

In [32]:
root = [-10, 18, 23, 27] #some base tuple, it will change as we go through the words since we are kinda only paying attention to the end of the word
circlelist = list(root) #list of the curvatures, it starts with the root stuff and we will add to it 
word = [1] #word that we are currently considering, it will change as we go through these, the first word we start with is just the word that consists of the first generator
recentroot = [] #this will be an array of arrays. the recentroot[i] is the most recent tuple that wasn't too big for word of length i so that when we go back down we know which tuple to operate on
recentroot.append(root) #the root thing will be our 0th element of this array
bound = 100000 #bound for what we want our curvatures to be under

while word: #this runs until you get back to the empty word
  #print("root : " + str(root))
  #print("word : " + str(word))
  n = word[-1] #we only really pay attention to the last generator in our word
  l = len(word) #so we know which tuple to pull from recentroot
  #print(recentroot)
  root = list(recentroot[l-1]) #you have to cast it as a list or else it is just referencing an element of recentroot and will change when that changes, which we don't want
  if transval(n,root)>bound: #if the new curvature was bigger than our bound, change the word according to the down function 
    word = down(word)
  else: #if it was not bigger than our bound, replace/append the corresponding element of recentroot, add that curvature to the circlelist, 
  #and change the word according to the over function, since we may go deeper into that part of the tree before we hit the bound
    root[n-1] = transval(n,root)
    if l>=len(recentroot):
      recentroot.append(root)
    else:
      recentroot[l]=root
    circlelist.append(root[n-1])
    word = over(word)

#print(circlelist)
print(len(circlelist))
curvefit(50,bound,circlelist) #this will print our fractal dimension once the circlelist has been made

66991
fractal dimension: 1.3049498628054264
